In [ ]:
from networks.UNet import LitUNet
from lightning import Trainer
from torch.nn.functional import softmax
import os
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
import torch
from glob import glob
from data.dataset import OEMDataset
from torch.utils.data import DataLoader
from pathlib import Path

In [ ]:
# map_location = {'cuda:0':'cpu'}
ckpt_pth = glob('/home/ubuntu/hrl/high-res-lulc/.neptune/Untitled/LUL-47/checkpoints/*.ckpt')
model = LitUNet.load_from_checkpoint(checkpoint_path=ckpt_pth[0], map_location='cpu')
trainer = Trainer(accelerator='gpu',num_nodes=1)

In [ ]:
DATA_DIR = "/home/ubuntu/hrl/high-res-lulc/data/processing"
VAL_LIST = os.path.join(DATA_DIR, "val.txt")
fns = [f for f in Path(DATA_DIR).rglob("*png") if "/images/" in str(f)]
val_list = [str(f)
            for f in fns
            if "_".join(f.name.split("_")[:-2]) + ".tif"
            in np.loadtxt(VAL_LIST, dtype=str)
        ]
val_list= val_list[:5]

In [ ]:
def pred_labels(trainer, dataloader):
    preds = trainer.predict(model, dataloaders=dataloader)
    # preds = torch.argmax(softmax(input=preds, dim=1), dim=1).permute(1,2,0).numpy()
    print(len(preds))
    
    
    return preds

In [ ]:

OEM_pred = DataLoader(OEMDataset(img_list=val_list, testing=False, augm=None))

In [ ]:
pred_labels = pred_labels(trainer=trainer, dataloader=OEM_pred)